### Load case reports

In [2]:
import json

file_in = open("dataset_articles.json")
papers = json.load(file_in)

def filter_papers(papers):
    res = []
    for paper in papers:
        if paper["abstract"] and len(paper["abstract"]) > 150 and paper["keywords"] not in [None,"."]:
            res.append(paper)
    return res

f_papers = filter_papers(papers)
len(papers), len(f_papers)

(864, 473)

### Calculate abstract pairs

In [3]:
import re

def process_keywords(keyword_s):
    keyword_s = keyword_s.lower().strip()
    keyword_s = re.sub(r"\\/","/", keyword_s)
    keywords = re.split(r"[,;/]",keyword_s)
    return [keyword.strip() for keyword in keywords]
s = "pnemumonia\/complicações, medicina"

process_keywords(s)

['pnemumonia', 'complicações', 'medicina']

In [4]:
from itertools import combinations
import re

paper_pairs = []
for paper1, paper2 in combinations(f_papers,2):
    #keywords1, keywords2 = process_keywords(paper1["keywords"]), process_keywords(paper2["keywords"])
    keywords1 = process_keywords(paper1["keywords"])
    keywords2 = process_keywords(paper2["keywords"])

    score = len( set(keywords1) & set(keywords2))
    paper_pairs.append((paper1["abstract"], paper2["abstract"], score))

len(paper_pairs)

111628

In [5]:
from collections import Counter
Counter([score for _,_,score in paper_pairs])

Counter({0: 110856, 1: 714, 2: 55, 3: 2, 4: 1})

### Balance similar and non-similar pairs

In [6]:
#Balancear dados
from sklearn.utils import resample
majority_class = [(a1, a2, score) for a1, a2, score in paper_pairs if score == 0]
minority_class = [(a1, a2, score) for a1, a2, score in paper_pairs if score != 0]

undersampled_majority_class = resample(majority_class,
                                       replace=False,     # Don't duplicate samples
                                       n_samples= len(minority_class),  # Match minority 
                                       random_state=42)

balanced_pairs = undersampled_majority_class + minority_class
Counter([score for _,_,score in balanced_pairs])

Counter({0: 772, 1: 714, 2: 55, 3: 2, 4: 1})

### Normalize similarity scores

In [7]:
def normalize_score(score):
    if score == 0:
        return 0
    if score == 1:
        return 0.5
    if score == 2:
        return 0.75
    if score >= 3:
        return 0.85    
balanced_pairs_norm = [(t1,t2,normalize_score(score)) for t1,t2,score in balanced_pairs]


### Generate test and train split

In [8]:
from sklearn.model_selection import train_test_split

scores = [score for _, _, score in balanced_pairs_norm]

train_data, test_data = train_test_split(
balanced_pairs_norm,
test_size=0.2,
random_state=42,
stratify=scores
)

score_train= Counter([score for _, _, score in train_data])
score_test= Counter([score for _, _, score in test_data])
print(score_train)
print(score_test)

Counter({0: 618, 0.5: 571, 0.75: 44, 0.85: 2})
Counter({0: 154, 0.5: 143, 0.75: 11, 0.85: 1})


In [9]:
from datasets import Dataset

def convert_to_dict_of_lists(data_tuples):

    result = {
        'abstract1': [],
        'abstract2': [],
        'score': [],
    }

    for abstract1, abstract2, score in data_tuples:
        result['abstract1'].append(abstract1)
        result['abstract2'].append(abstract2)
        result['score'].append(score)

    return result

train_dataset = Dataset.from_dict(convert_to_dict_of_lists(train_data))
test_dataset = Dataset.from_dict(convert_to_dict_of_lists(test_data))


# Check the datasets
print("Train Dataset:", train_dataset)
print("Validation Dataset:", test_dataset)


/home/lfc/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train Dataset: Dataset({
    features: ['abstract1', 'abstract2', 'score'],
    num_rows: 1235
})
Validation Dataset: Dataset({
    features: ['abstract1', 'abstract2', 'score'],
    num_rows: 309
})


### Train the Bi-encoder model

In [15]:
from sentence_transformers import SentenceTransformer, losses


model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
loss= losses.CosineSimilarityLoss(model)

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.
/home/lfc/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator



args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/my_model",
    report_to="none",
    # Optional training parameters:
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)


# Create the evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    test_dataset['text1'],  # Assuming these are the sentence pairs for evaluation
    test_dataset['text2'],
    test_dataset['score'],  # Assuming this contains the similarity scores
    main_similarity=SimilarityFunction.COSINE,
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)



In [ ]:
trainer.train()

### Inference

In [16]:
model = SentenceTransformer("lfcc/medlink-bi-encoder")

In [17]:
abstracts = [entry["abstract"] for entry in f_papers]
titles = [entry["title"] for entry in f_papers]
keywords = [entry["keywords"] for entry in f_papers]

In [19]:
query = """S/
Identidicação: género feminino, 24 anos.

AP:
# sem antecedentes conhecidos.

MH:
# contraceptivo oral

HDA: Recorre ao serviço de urgência por nódulos violáceos dolorosos na região pré-tibial bilateral, que estenderam a toda a perna e coxa, com 1 mês de evolução. Três semanas antes do início do quadro, realizou a 2ª dose da vacina contra SARS-CoV-2 Comirnaty® (Pfizer-BioNTech), a mesma que realizou um mês antes. Sem outras queixas, nomeadamente sugestivas de patologia auto-imune ou síndrome onstitucional.

O/
Exame físico- apirética, sem alterações da orofaringe ou adenopatias, observando-se lesões nodulares em ambas as coxas e pernas, de cor violácea, eritematosas, com 2 cm de maior eixo, dolorosas à palpação.

A/
# Analiticamente- leucocitose de 12,6 x 109/L com neutrofilia, proteína C reactiva 210 mg/L e velocidade de sedimentação 42 mm/h.
# Anticorpo anti-estreptolisina O ne-gativo, serologias para sífilis e vírus Ebstein-Barr, hepatite B e C e VIH negativas. Painel imunológico, com pesquisa de anticorpos anti-nucleares (ANA), anticorpos anti-double--stranded DNA (dsDNA), anticorpos itoplasmáticos anti--neutrófilos (ANCA), complemento, factor reumatóide e anticorpos anti-péptido citrulinado cíclico, negativos.
# Marcadores de lesão hepática e função renal sem alterações.
# Hemoculturas negativas.
# TC TAP sem alterações.

Assim, assumido eritema nodoso secundário à administração da vacina contra SARS-CoV-2.
Iniciada prednisolona 60 mg/dia.

Evolução:
# Uma semana depois houve melhoria clínica significativa e diminuição acentuada dos parâmetros inflamatórios, pelo que se iniciou redução gradual de corticoterapia, com resolução completa do quadro ao fim de duas semanas.
# Passados seis meses, a doente teve doença ligeira por SARS-CoV-2 e uma semana depois surgiram lesões com características idênticas às do episódio anterior. Iniciou novo ciclo de corticoterapia, com resolução do quadro. Pela exuberância das lesões, a doente foi aconselhada a não repetir imunização com vacina de mRNA contra SARS-CoV-2.
# Até à data, não voltou ter infecção por SARS-CoV-2 nem teve ressurgimento das lesões cutâneas.
"""

In [18]:
abstract_embeddings = model.encode(abstracts)

In [21]:
from sentence_transformers import util
import torch

query_embedding = model.encode(query)

# Calculate the similarity between the query and the abstracts
cosine_scores = util.pytorch_cos_sim(query_embedding, abstract_embeddings)
retrieval_results = torch.topk(cosine_scores, k=15)

for value, index in zip(retrieval_results.values[0], retrieval_results.indices[0]):
    print("Titulo: ", titles[index])
    print("Keywords: ", keywords[index])
    print("Score: ", value )
    print("_"*100)


Titulo:  Um Caso Clínico de Síndrome de Evans Provavelmente Desencadeada por Vacina BNT162b2 SARS-CoV-2
Keywords:  Anemia Hemolítica Autoimune/induzida quimicamente, Vacina BNT162b2, Vacinas contra COVID-19
Score:  tensor(0.6316)
____________________________________________________________________________________________________
Titulo:  Púrpura Trombocitopénica Imune Após Vacina SARS-CoV-2 da Moderna®: Um Estudo de Caso
Keywords:  COVID-19, Púrpura Trombocitopênica Idiopática/induzida quimicamente, SARS-CoV-2, Vacina de mRNA-1273 contra 2019-nCoV, Vacinas contra COVID-19/efeitos adversos
Score:  tensor(0.6271)
____________________________________________________________________________________________________
Titulo:  DISFUNÇÃO MULTIORGÂNICA PARA ALÉM DA SÉPSIS: LINFOHISTIOCITOSE HEMOFAGOCÍTICA.
Keywords:  Disfunção multiorgânica; Linfohistiocitose hemofagocítica; Linfoma de células B; Síndrome de Ativação Macrofágica; Sépsis
Score:  tensor(0.5970)
____________________________________